In [35]:
import numpy as np
import pandas as pd

In [36]:
# google drive data mount 
funda_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/funda_train.csv")
submission = pd.read_csv("/content/drive/My Drive/Colab Notebooks/submission.csv")

In [37]:
funda_train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.14286
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.14286
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.00000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.14286
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.00000


In [38]:
funda_train.shape

(6556613, 9)

In [39]:
 submission.head()

,store_id,amount
0,0,0
1,1,0
2,2,0
3,4,0
4,5,0


In [40]:
submission.shape

(1967, 2)

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
# default
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from sklearn.metrics import make_scorer

# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# NN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time

%matplotlib inline
pd.options.display.max_columns = 400
pd.options.display.float_format = '{:.5f}'.format

In [98]:
# 평가척도 
def mae(prediction, correct):
  # for shape
  prediction = np.array(prediction)
  correct = np.array(correct)

  diff = correct - prediction
  abs_val = abs(diff)

  score = abs_val.mean()

  return score

In [44]:
# matric object
mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

In [45]:
# for time data col
funda_train['transacted_date'] = pd.to_datetime(funda_train['transacted_date'])

time series analysis

In [46]:
# date정보를 idx로 반환한다
df_train = funda_train.set_index('transacted_date')

In [47]:
df_train.head(3)

,store_id,card_id,card_company,transacted_time,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,
2016-06-01,0,0,b,13:13,0,NaN,기타 미용업,1857.14286
2016-06-01,0,1,h,18:12,0,NaN,기타 미용업,857.14286
2016-06-01,0,2,c,18:52,0,NaN,기타 미용업,2000.00000


In [48]:
df_train.shape

(6556613, 8)

In [52]:
# 시간단위 데이터를 일단위로 resample진행
# 일별로 읽을 수 있는 새로운 feature생성 
def resample_day(train_df):
    df_day = pd.DataFrame()         # 최종 반환할 데이터프레임 
    
    for i in train_df.store_id.unique():
        df_num = train_df[train_df.store_id == i]

        count_cols = df_num['card_id'].resample(rule='d').count().rename('num_of_pay') # 'card_id' 의 일별 counting을 통해 일 거래 횟수 확인
        # 'card_id' value count가 2보다 크면 단골인 것으로 판단하고 단골 방문 횟수 확인
        revisit_idx = df_num.card_id.value_counts().reset_index().query("card_id > 2")["index"].values 
        revisit_ct = df_num[df_num.card_id.isin(revisit_idx)].card_id.resample(rule='d').count().rename('num_of_revisit')  
        sum_cols = df_num[['installment_term', 'amount']].resample(rule='d').sum() # 할부 개월수와 매출액은 일 단위로 합

        df_num_day = pd.concat([count_cols, revisit_ct, sum_cols], axis=1)

        df_num_day.insert(0, 'store_id', i)
        df_num_day.insert(4, 'region', df_num[df_num.store_id == i].region.unique()[0])
        df_num_day.insert(5, 'type_of_business', df_num[df_num.store_id == i].type_of_business.unique()[0])

        df_day = pd.concat([df_day, df_num_day], axis=0)
        
    df_day.insert(1, 'day_of_week', df_day.index.dayofweek)
    df_day.insert(2, 'business_day', df_day.day_of_week.replace({0:1, 2:1, 3:1, 4:1, 5:0, 6:0}).values)
    df_day.num_of_revisit.fillna(0, inplace=True)
    
    return df_day

In [53]:
# 확인
%%time
df_day = resample_day(df_train)

CPU times: user 3min 19s, sys: 4.82 s, total: 3min 23s
Wall time: 3min 24s


In [54]:
df_day.shape

(1839261, 9)

In [55]:
# 만든 데이터 확인
df_day.head(3)

,store_id,day_of_week,business_day,num_of_pay,num_of_revisit,installment_term,region,type_of_business,amount
transacted_date,,,,,,,,,
2016-06-01,0,2,1,4,4.00000,0,NaN,기타 미용업,12571.42857
2016-06-02,0,3,1,7,3.00000,0,NaN,기타 미용업,40571.42857
2016-06-03,0,4,1,3,2.00000,0,NaN,기타 미용업,18142.85714


In [56]:
# null값 처리 
df_day.isnull().sum()

store_id                  0
day_of_week               0
business_day              0
num_of_pay                0
num_of_revisit            0
installment_term          0
region               629927
type_of_business    1087791
amount                    0
dtype: int64

In [57]:
# 파일 저장 
df_day.to_csv("/content/drive/My Drive/Colab Notebooks/funda_train_day.csv")

In [58]:
df_day = pd.read_csv("/content/drive/My Drive/Colab Notebooks/funda_train_day.csv") # 불러와서 파일 사용 및 처리 
df_day['transacted_date'] = pd.to_datetime(df_day['transacted_date'])
df_day = df_day.set_index('transacted_date')

In [59]:
# 각 col상관관계 확인 
daily_corr = df_day.corr()
daily_corr

,store_id,day_of_week,business_day,num_of_pay,num_of_revisit,installment_term,amount
store_id,1.00000,-0.00004,0.00001,0.04237,0.00806,-0.01218,-0.01835
day_of_week,-0.00004,1.00000,-0.79052,0.01672,-0.03869,-0.01395,-0.02150
business_day,0.00001,-0.79052,1.00000,-0.00460,0.05721,0.01530,0.02947
num_of_pay,0.04237,0.01672,-0.00460,1.00000,0.70112,0.09414,0.26525
num_of_revisit,0.00806,-0.03869,0.05721,0.70112,1.00000,0.07113,0.26067
installment_term,-0.01218,-0.01395,0.01530,0.09414,0.07113,1.00000,0.26750
amount,-0.01835,-0.02150,0.02947,0.26525,0.26067,0.26750,1.00000


In [73]:
# 다시 month단위로 resample
def resample_month(frame_day):
  sum_cols = ['num_of_pay', 'num_of_revisit', 'installment_term', 'amount']
  df_monthly = pd.DataFrame() # 새롭게 배정할 데이터프레임 설정

  for i in frame_day.store_id.unique():
    df_set = frame_day[frame_day.store_id == i] # 각 상점의 id가 같은 것끼리 묶는다

    # nan값이 발생하는 경우를 없애기 위해 이전, 이후 달에 대한 정보를 추가한 후 제거
    prev_date = pd.date_range(start=(df_set.index[0] - relativedelta(months=1)), end=(df_set.index[0] - relativedelta(months=1)))
    add_date = pd.date_range(start=(df_set.index[-1] + relativedelta(months=1)), end=(df_set.index[-1] + relativedelta(months=1)))
    df_set = pd.concat([pd.DataFrame(index=prev_date), df_set, pd.DataFrame(index=add_date)], axis=0)

    df_set.loc[dt.strftime(df_set.index[0], '%Y-%m'), :] = 1
    df_set.loc[dt.strftime(df_set.index[-1], '%Y-%m'), :] = 1

    tot_day = df_set[df_set.amount != 0].day_of_week.resample(rule='m').count().rename('real_tot_day')
    business = df_set[df_set.amount != 0].business_day.resample(rule='m').sum().rename('real_business_day')

    business = business.drop([business.index[0], business.index[-1]], axis=0)
    tot_day = tot_day.drop([tot_day.index[0], tot_day.index[-1]], axis=0)
    df_set = df_set.drop([df_set.index[0], df_set.index[-1]], axis=0)

    df = pd.concat([tot_day, business, df_set[sum_cols].resample(rule='m').sum()], axis=1)

    df.insert(0, 'store_id', i)
    df.insert(6, 'region', df_set.region.values[0])
    df.insert(7, 'type_of_business', df_set.type_of_business.values[0])

    df_monthly = pd.concat([df_monthly, df], axis=0)
   
  return df_monthly

In [74]:
%%time
df_month = resample_month(df_day)

CPU times: user 1min 10s, sys: 287 ms, total: 1min 10s
Wall time: 1min 10s


In [75]:
# null값 확인
df_month.isnull().sum()

store_id                 0
real_tot_day             0
real_business_day        0
num_of_pay               0
num_of_revisit           0
installment_term         0
region               20900
type_of_business     36048
amount                   0
dtype: int64

In [76]:
df_month.shape

(60915, 9)

In [77]:
df_month.head(3)

,store_id,real_tot_day,real_business_day,num_of_pay,num_of_revisit,installment_term,region,type_of_business,amount
2016-06-30,0,25,17.00000,145.00000,77.00000,13.00000,NaN,기타 미용업,747000.00000
2016-07-31,0,26,16.00000,178.00000,105.00000,24.00000,NaN,기타 미용업,1005000.00000
2016-08-31,0,24,16.00000,171.00000,97.00000,69.00000,NaN,기타 미용업,871571.42857


In [78]:
# null값이 많은 두 col의 비율 확인
print("region", df_month.region.isnull().sum() / len(df_month.region))
print("type_of_bissiness:",  df_month.type_of_business.isnull().sum() / len(df_month.type_of_business))

region 0.34310104243618156
type_of_bissiness: 0.5917754247722236


In [79]:
# null값의 비중이 높은 col은 삭제한다. 
df_month = df_month.drop(['region', 'type_of_business'], axis = 1)

In [80]:
df_month.isnull().sum()

store_id             0
real_tot_day         0
real_business_day    0
num_of_pay           0
num_of_revisit       0
installment_term     0
amount               0
dtype: int64

In [81]:
# month딘위 resample결과 저장 
df_month.to_csv("/content/drive/My Drive/Colab Notebooks/funda_train_month.csv")

In [82]:
# month 단위 resample 결과 불러와 사용
df_month = pd.read_csv("/content/drive/My Drive/Colab Notebooks/funda_train_month.csv")
df_month.rename(columns={'Unnamed: 0' : 'transacted_date'}, inplace=True)
df_month['transacted_date'] = pd.to_datetime(df_month['transacted_date'])
df_month = df_month.set_index('transacted_date')

In [83]:
df_month.head(3)

,store_id,real_tot_day,real_business_day,num_of_pay,num_of_revisit,installment_term,amount
transacted_date,,,,,,,
2016-06-30,0,25,17.00000,145.00000,77.00000,13.00000,747000.00000
2016-07-31,0,26,16.00000,178.00000,105.00000,24.00000,1005000.00000
2016-08-31,0,24,16.00000,171.00000,97.00000,69.00000,871571.42857


In [84]:
df_month.shape # (5~, 7) col임을 확인 

(60915, 7)

In [85]:
# 변수 기초통계량
df_month.describe()

,store_id,real_tot_day,real_business_day,num_of_pay,num_of_revisit,installment_term,amount
count,60915.00000,60915.00000,60915.00000,60915.00000,60915.00000,60915.00000,60915.00000
mean,1050.48428,21.06987,15.89640,107.63544,40.01136,14.87540,1123187.83063
std,617.48320,7.75327,5.63136,101.24413,43.88133,33.42161,1753567.15171
min,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-514285.71429
25%,515.00000,16.00000,12.00000,32.00000,11.00000,0.00000,339642.85714
50%,1039.00000,23.00000,17.00000,73.00000,26.00000,3.00000,710714.28571
75%,1587.00000,28.00000,21.00000,159.00000,54.00000,14.00000,1344650.00000
max,2136.00000,31.00000,23.00000,2186.00000,2154.00000,1125.00000,61429316.14286


In [87]:
# target변수의 대칭성 확인 (target = amount value)
# 각 store_id의 3개의 값에 대해 대칭성을 먼저 판단하고 ( 기준 1) 
for i in df_month.store_id.unique()[:3]:
  print("왜도: ", df_month[df_month.store_id == i].amount.skew())

왜도:  0.48245510908400735
왜도:  0.5102521050694732
왜도:  0.9982599271676977


## modeling
- 1. simple moving average
- 2. exponential moving average
- 3. exponential smoothing( holt's, holt's winter)
- 4. ARMA
- 5. ARIMA

Simple moving average

In [93]:
def make_sma_arr(window_num):
  ma_arr = np.array([]) # moving average에 해당하는 A_i값의 평균값 저장 array, return vlaue 

  for i in df_month.store_id.unique():
    df_set = df_month[df_month.store_id == i] # 마찬가지로 store_id에 따라 묶어서 진행
    ma_arr = np.concatenate((ma_arr, df_set.amount.rolling(window=window_num).mean().values))
  return ma_arr

In [101]:
sma_month = df_month.copy()

In [102]:
sma_month.insert(7, 'amount_2ma', make_sma_arr(2))
sma_month.insert(8, 'amount_3ma', make_sma_arr(3))
sma_month.insert(9, 'amount_6ma', make_sma_arr(6))

In [103]:
sma_month.head(3)

,store_id,real_tot_day,real_business_day,num_of_pay,num_of_revisit,installment_term,amount,amount_2ma,amount_3ma,amount_6ma
transacted_date,,,,,,,,,,
2016-06-30,0,25,17.00000,145.00000,77.00000,13.00000,747000.00000,nan,nan,nan
2016-07-31,0,26,16.00000,178.00000,105.00000,24.00000,1005000.00000,876000.00000,nan,nan
2016-08-31,0,24,16.00000,171.00000,97.00000,69.00000,871571.42857,938285.71429,874523.80952,nan


In [104]:
# metric
print("2 winodw SMA MAE Score : ", mae(sma_month.amount_2ma.fillna(0), sma_month.amount.values))
print("3 window SMA MAE Score : ", mae(sma_month.amount_3ma.fillna(0), sma_month.amount.values))
print("6 window SMA MAE Score : ", mae(sma_month.amount_6ma.fillna(0), sma_month.amount.values))

2 winodw SMA MAE Score :  180687.25906004856
3 window SMA MAE Score :  251607.45528312284
6 window SMA MAE Score :  387465.0993765708


In [115]:
# rolling에 대한 t시점의 평균을 지정한 window에 따라 average를 설정해야 한다.
# amount값을 조정하여 잎의 3달의 값을 예측한다  
def make_minus_rolling(data_frame, rolling_num):
    def minus_shift_rolling(df_num, num):
        a = np.average(df_num.values[-num:])
        b = np.average(np.append(df_set.values[-(num-1):], a))
        if num > 2:
            c = np.average(np.append(np.append(df_set.values[-(num-2):], a), b))
        else:
            c = np.average((a, b))
        return np.sum((a, b, c))

    minus_rolling_arr = np.array([])
    for i in data_frame.store_id.unique():
        df_set = pd.DataFrame(data_frame[data_frame.store_id == i].amount)
        minus_rolling_arr = np.concatenate((minus_rolling_arr, np.array([minus_shift_rolling(df_set, rolling_num)])))
        
    submission['amount'] = minus_rolling_arr
    df_rolling = submission
    
    return df_rolling

In [116]:
for i in range(2, 5):
    df_rolling = make_minus_rolling(df_month, i)
    df_rolling.to_csv('funda_{}_rolling_sub_re.csv'.format(i), index=False)

expoential moveing average

In [118]:
def make_ewm_arr(data_frame, span_num):
  arr_ewm = np.array([]) #반환할 expoential data for month

  for i in data_frame.store_id.unique():
    df_set = data_frame[data_frame.store_id == i]
    # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.amount.ewm(span=span_num).mean().values))
    
  return arr_ewm

In [122]:
ewm_month = df_month.copy()

In [123]:
ewm_month.insert(7, 'amount_3ewm', make_ewm_arr(df_month, 3))
ewm_month.insert(8, 'amount_6ewm', make_ewm_arr(df_month, 6))

In [124]:
print("3 N EWM MAE Score : ", mae(ewm_month.amount_3ewm, sma_month.amount.values))
print("6 N EWM MAE Score : ", mae(ewm_month.amount_6ewm, sma_month.amount.values))

3 N EWM MAE Score :  134855.11946915495
6 N EWM MAE Score :  193447.63093078104


In [131]:
# exponential weighted moving score확인 
def make_wma_sub(data_frame, span_num):
    concat_3mon = pd.DataFrame(index=pd.to_datetime(['2019-03-31', '2019-04-30', '2019-05-31']))
    wma_sub = np.array([])

    for i in df_month.store_id.unique():
        df_set = pd.DataFrame(data_frame[data_frame.store_id == i].amount)
        wma_train = pd.concat([df_set, concat_3mon], axis=0)

        num_sub = np.array([wma_train.amount.ewm(span=span_num).mean()['2019-03':].sum()])

        wma_sub = np.concatenate((wma_sub, num_sub))
    
    submission['amount'] = wma_sub
    df_wma_sub = submission
    
    return df_wma_sub

In [132]:
for i in range(2, 7):
    wma_sub = make_wma_sub(df_month, i)
    wma_sub.to_csv('funda_{}wma_sub.csv'.format(i), index=False)